SVM example: Identifyng Chronic Kidney Disease
----

# Introduction

This notebook walks through an example implementing a Support Vector Machine to tackle a medical classification problem. The data comes from the UCI Machine Learning Repository (https://archive.ics.uci.edu/ml/datasets/Chronic_Kidney_Disease), based on research by P. Soundarapandian, L. Jerlin Rubini, and P. Esweran.


## Outline

1. Implementing a linear SVM
1. Applying and tuning the model
1. Implementing and applying kernel SVM

## Setup

We will be using both [NumPy](https://numpy.org/doc/stable/index.html) arrays and [pandas](https://pandas.pydata.org/docs/index.html) DataFrames, so we need to import these packages with their standard abbreviations.

In [147]:
import numpy as np
import pandas as pd

RNG = np.random.default_rng()

# 1. Implementing a Linear SVM

## Theory

Recall that a **Support Vector Machine** seeks to minimize classification errors while maximizing the width of the margin around the decision boundary, in hopes of increasing generalizability. More formally, we have a dataset of $n$ feature vectors $x^{(i)}$ and labels $y^{(i)}\in {-1,1}$. We want to find a set of parameters ($\theta, \theta_0$) that mimimize the cost function:
$$ J(\theta , \theta _0) = \frac{1}{n} \sum _{i=1}^{n} \text {Loss}_ h (y^{(i)} (\theta \cdot x^{(i)} + \theta _0 )) + \frac{\lambda }{2} \mid \mid \theta \mid \mid ^2$$
where
$$\text{Loss}_h(z)=\begin{cases}0 & z \ge 1 \\1-z & z < 1\end{cases}$$
and $\lambda$ is the **regularization parameter**, determining the relative weight given to margin width (i.e. the inverse of the squared norm of $\theta$) and accuracy (i.e. the `Loss` term). Higher $\lambda$ focuses more on regularization by widening the margin.

We estimate the parameters through **Stochastic Gradient Descent (SGD)**. This means we randomly select an observation $i$ and update the parameters as follows:

$$\theta \leftarrow \theta - \eta \nabla _{\theta } \big [\text {Loss}_ h(y^{(i)}(\theta \cdot x^{(i)} + \theta _0) ) + \frac{\lambda }{2}\mid \mid \theta \mid \mid ^2 \big ]$$
where $\nabla$ is the learning rate. This can be a constant or can be adjusted over the training process.

If the case is placed correctly and outside the classification margin by the current parameters (i.e. $y^{(i)}(\theta \cdot x^{(i)} + \theta _0)\ge 1$), the update is based strictly on the regularization term. So, the gradient is $\lambda \theta$.

If there is a positive `Loss` (i.e. $y^{(i)}(\theta \cdot x^{(i)} + \theta _0)<1$), then the gradient is $-y^{(i)}x^{(i)} + \lambda \theta$.

We continue cycling through the data, shuffling after each epoch, until the change in the cost remains below some threshold.

Note that this algorithm only optimizes the parameters $\theta$, and $\theta_0$, while $\lambda$, $\nabla$, and the convergence threshold are **hyperparameters** that must be set in advance.

NOTE ABOUT THE OFFSET: rather than treating $\theta_0$ as a separate parameter, it is possibe instead to prepend a `1` to every feature vector, so that the SVM trains $\theta$ as a $D+1$ vector, where the first entry is equivalent to $\theta_0$. This is what will be done below.

## Implementation

The cell below implements a linear SVM as a class that mimics the syntax of the popular machine learning package [scikit-learn](https://scikit-learn.org/stable/index.html).
1. The model is first initialized with given (default or custom) hyperparameters. Here those are the regularization paramter `lam` (for $\lambda$), the learning rate `lrate`, and the convergence `threshold`.
1. Then, the model is trained on data using the method `.fit(X, y)`, where X contains the features vectors (observations as rows, features as columns) and y contains the labels. The model saves the trained parameters.
1. After that, the model can be used to assign labels to any feature vectors with the method `.predict(X)`
1. Finally, `.score(X, y)` calculates and returns the share of correctly predicted cases.

The class below also contains three helper functions called by `.predict()`:
1. `.cost()` calculates the cost function for the entire dataset, needed to check for convergence
1. `.loss()` calculates and returns the Loss function for calculating the cost function, again as defined above
1. `.train_epoch()` runs through the observations in random order one time, updating the parameters accordingly, and returning the resulting cost function.

In [148]:
class LinearSVM():
    def __init__(self, lam=0.5, lrate=1., threshold=1e-7) -> None:
        '''
        Initialize the model with the given hyperparameters:
        - `lam`: the regularization parameter lambda
        - `lrate`: the learning rate
        - `threshold`: the convergence threshold
        '''
        self.lam = lam
        self.lrate = lrate
        self.threshold = threshold
        self.theta = None
        self.max_iter = 1000

    def loss(self, Xi, yi):
        '''The Loss function for one observation'''
        if self.theta is None:
            raise ValueError('Cannot calculate loss without initializing theta')

        agreement = yi*(self.theta @ Xi)
        return 0 if agreement >= 1 else 1-agreement
    
    def cost(self, X, y):
        '''Average cost for the whole dataset'''
        loss_sum = 0
        for Xi, yi in zip(X, y):
            loss_sum += self.loss(Xi, yi)
        return loss_sum/len(y) + self.lam/2 * (self.theta@self.theta)
    
    def train_epoch(self, X, y):
        '''Cycles through cases in random order, updating theta. Returns the average cost after every update'''
        order = np.arange(len(y)) 
        RNG.shuffle(order)
        for i in order:
            grad = self.lam * self.theta
            grad -= y[i]*X[i] if self.loss(X[i], y[i]) > 0 else 0
            self.theta -= self.lrate * grad
        return self.cost(X, y)
    
    def fit(self, X, y):
        '''Fits the model to the data (feature vectors X, labels y). To fit with an offset, prepend `1` to every feature vector'''
        # initialize theta
        self.theta = np.zeros(X.shape[-1])

        last_cost = np.inf
        for i in range(self.max_iter):
            new_cost = self.train_epoch(X, y)
            if np.abs(last_cost - new_cost) < self.threshold:
                break
            else:
                last_cost = new_cost
                self.lrate = self.lrate/(1+i)
        else:
            print(f"Maximum of {self.max_iter} reached without convergence")
        return self
    
    def predict(self, X):
        '''Predicts labels for feature vectors X'''
        return sign(X @ self.theta)
    
    def score(self, X, y):
        y_pred = self.predict(X)
        return np.mean(y == y_pred)

@np.vectorize
def sign(z):
    '''Converts values to labels, i.e. 1 and -1'''
    return 1 if z>=0 else -1

# 2. Applying and Tuning the Model

The data is stored as in the file `ckd_data.csv`, and we can read it into a pandas DataFrame.

The data has 400 rows. However, only the target label (`class`) has a full 400 observations: every feature has at least some missing values.

In [149]:
raw_data = pd.read_csv('ckd_data.csv', index_col=0)
print(raw_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400 entries, 0 to 399
Data columns (total 25 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     391 non-null    float64
 1   bp      388 non-null    float64
 2   sg      353 non-null    float64
 3   al      354 non-null    float64
 4   su      351 non-null    float64
 5   rbc     248 non-null    object 
 6   pc      335 non-null    object 
 7   pcc     396 non-null    object 
 8   ba      396 non-null    object 
 9   bgr     356 non-null    float64
 10  bu      381 non-null    float64
 11  sc      383 non-null    float64
 12  sod     313 non-null    float64
 13  pot     312 non-null    float64
 14  hemo    348 non-null    float64
 15  pcv     329 non-null    float64
 16  wbcc    294 non-null    float64
 17  rbcc    269 non-null    float64
 18  htn     398 non-null    object 
 19  dm      398 non-null    object 
 20  cad     398 non-null    object 
 21  appet   399 non-null    object 
 22  pe

As a first pass, we can adopt the most conservative approach to the missing values: dropping any row that contained any. This results in 158 valid observations.

In [150]:
data_dropNA = raw_data.dropna()
data_dropNA.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 3 to 399
Data columns (total 25 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     158 non-null    float64
 1   bp      158 non-null    float64
 2   sg      158 non-null    float64
 3   al      158 non-null    float64
 4   su      158 non-null    float64
 5   rbc     158 non-null    object 
 6   pc      158 non-null    object 
 7   pcc     158 non-null    object 
 8   ba      158 non-null    object 
 9   bgr     158 non-null    float64
 10  bu      158 non-null    float64
 11  sc      158 non-null    float64
 12  sod     158 non-null    float64
 13  pot     158 non-null    float64
 14  hemo    158 non-null    float64
 15  pcv     158 non-null    float64
 16  wbcc    158 non-null    float64
 17  rbcc    158 non-null    float64
 18  htn     158 non-null    object 
 19  dm      158 non-null    object 
 20  cad     158 non-null    object 
 21  appet   158 non-null    object 
 22  pe

Next, we need to convert the non-numeric columns (dtype object above) into a numeric format. It turns out that each of these features represent binary categories. So, we can easily use binary encoding: replacing the categorical feature with that is 1 if the case falls within one category, otherwise 0. This can be done with the pandas method `.getdummies()`.

In [151]:
print(data_dropNA.select_dtypes('object').apply(pd.unique))
data_dropNA = pd.get_dummies(data_dropNA, drop_first=True)
data_dropNA.info()

        rbc        pc         pcc          ba  htn   dm  cad appet   pe  ane  \
0    normal  abnormal     present  notpresent  yes   no   no  poor  yes  yes   
1  abnormal    normal  notpresent     present   no  yes  yes  good   no   no   

    class  
0     ckd  
1  notckd  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 3 to 399
Data columns (total 25 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   age           158 non-null    float64
 1   bp            158 non-null    float64
 2   sg            158 non-null    float64
 3   al            158 non-null    float64
 4   su            158 non-null    float64
 5   bgr           158 non-null    float64
 6   bu            158 non-null    float64
 7   sc            158 non-null    float64
 8   sod           158 non-null    float64
 9   pot           158 non-null    float64
 10  hemo          158 non-null    float64
 11  pcv           158 non-null    float64
 12  wbcc       

For the label, we can rename and recode for easier interpretability: the positive label $+1$ applies to patients with chronic kidney disease, and the negative label $-1$ to those who do not.  

In [152]:
data_dropNA.rename(columns={'class_notckd':'is_ckd'}, inplace=True)
data_dropNA['is_ckd'] = data_dropNA.is_ckd*-2 + 1
data_dropNA.is_ckd.value_counts()

-1    115
 1     43
Name: is_ckd, dtype: int64

Finally, we need to separate the feature vectors from the labels and split our data into training and test sets. The training data is used to train the model, the test data is used to assess its generalizable performance. We will set aside 20% of the latter as the test data. This is done randomly, with one adjustment: we want to preserve the rough balance of positively and negatively labeled cases in the training and testing data.

We thus will end up with 4 numpy arrays: `X_train`, `X_test`, `y_train`, `y_test`.

In [153]:
X = data_dropNA.drop(columns='is_ckd').to_numpy()
y = data_dropNA.is_ckd.to_numpy()

train_share = 0.8

positive_indices = np.flatnonzero(y == 1)
RNG.shuffle(positive_indices)
n_train_positive = int(train_share * positive_indices.shape[0])

negative_indices = np.flatnonzero(y == -1)
RNG.shuffle(negative_indices)
n_train_negative = int(train_share * len(negative_indices))

train_indices = np.concatenate(
    [
        positive_indices[:n_train_positive], 
        negative_indices[:n_train_negative]
    ], 
    axis=None)
test_indices = np.concatenate(
    [
        positive_indices[n_train_positive:], 
        negative_indices[n_train_negative:]
    ], 
    axis=None)

X_train = X[train_indices]
X_test = X[test_indices]
y_train = y[train_indices]
y_test = y[test_indices]

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(126, 24) (126,) (32, 24) (32,)


In [154]:
default_svm = LinearSVM()
default_svm.fit(X_train, y_train)
default_svm.score(X_test, y_test)

138866396.4501971
352578.7102546689
924884.805999862
4856963.899036874
173523.70305159467
84489.90593604438
26773.477902543167
363.4109548462469
348.31436751412804
343.4831582728844
336.5929340539032
336.08879765203363
336.0467863574888
336.0435547199029
336.04332388865146
336.04330849990174
336.0433075381046
336.0433074815296


0.9375